In [23]:
from Gridworld import Gridworld
# Existen 3 modos de juego:
# 'static' <-- Todo inicia en las mismas posiciones predeterminadas
# 'player' <-- Solo el jugador inicia en una posicion aleatoria
# 'random' <-- Todos los objetos inician de forma aleatoria
# size es el tamaño del tablero n x n
game = Gridworld(size=4, mode='static')

In [24]:
# Asi se puede mostrar graficamente el ambiente del juego
game.display()

array([['+', '-', ' ', 'P'],
       [' ', 'W', ' ', ' '],
       [' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ']], dtype='<U2')

In [25]:
# Asi se hace una jugada 
# 'u' <--- arriba
# 'd' <--- abajo
# 'l' <--- izquierda
# 'r' <--- derecha
game.makeMove('d')
game.display()

array([['+', '-', ' ', ' '],
       [' ', 'W', ' ', 'P'],
       [' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ']], dtype='<U2')

In [28]:
# Asi se muestra la recompensa obtenida
# + 10 por ganar '+'
# - 10 por caer al pozo '-'
# -1 por jugada sin ganar
# 'w' es un muro
game.reward()

-1

In [27]:
# Asi se representa el juego
# Cada matriz codifica la posición de uno de los cuatro objetos: 
# el jugador, la portería, el foso y el muro.
game.board.render_np()

array([[[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

In [29]:
# Es un tensor de forma (4, 4, 4)
game.board.render_np().shape

(4, 4, 4)

In [ ]:
import numpy as np
